<a target="_blank" href="https://colab.research.google.com/github/HerQTheToxic/DeepL/blob/main/Detection.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#Image path
imagePath="/content/Test.png"

In [ ]:
#YOLOv5 dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt comet_ml

In [ ]:
import argparse
import cv2
import torch
import os
import logging
import sys
from io import StringIO

In [ ]:
#Downloading trained model weights (stored in my public drive)
!gdown https://drive.google.com/uc?id=1a59RBjKrMXo2szDmEELmzsnAlg7aW5un -O /content/Number.pt
!gdown https://drive.google.com/uc?id=1BpJhTWsGL6ZvtPrDqbGiYV4DJXMW0QoF -O /content/Gem2.pt
!gdown https://drive.google.com/uc?id=1jmatSFxw_TgjMyowckzvDAKBx50HPNOz -O /content/GemCls.pt
!gdown https://drive.google.com/uc?id=1XZN4jWr1T36iaoXDllla3bJuTEvj8SwK -O /content/Test.png

In [ ]:
#YOLOv5 framework path
yolov5Path = "/content/yolov5"

#modell weight paths
NumberModelPath="/content/Number.pt"
GemModelPath="/content/Gem2.pt"
GemModelClassPath="/content/GemCls.pt"

In [ ]:
#Detecting with the modell
def detectx(frame, model):
    results = model([frame])
    labels, coordinates = results.pred[0][:, -1], results.pred[0][:, :-1]
    return labels, coordinates

In [ ]:
#Loading model with the trained weight
def loadModel(path):
    model = torch.hub.load(yolov5Path, 'custom', source='local', path=path, force_reload=True, skip_validation=True)
    return model

In [ ]:
#Converting image into useable frame for detection
def getFrame(image_path):
    frame = cv2.imread(image_path)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame

In [ ]:
#Finding the numbers in the picture. Sorting in order by X coordinate and converting to final number
def getDetectedNumber(frame, model):
        #Getting the numbers
    detectedNumbers = []

    labelsNum, coordinatesNum = detectx(frame, model)
    class_names_Num = model.names

    for i in range(len(labelsNum)):
            label = labelsNum[i]
            coordinate = coordinatesNum[i]
            if coordinate[4] >= threshold:
                class_label = class_names_Num[int(label)]
                x1, y1, x2, y2 = coordinate[:4]
                detectedNumbers.append((class_label, x1))
    if len(detectedNumbers) < 1:
        return 0
    else:
        detectedNumbers = sorted(detectedNumbers, key=lambda x: x[1])

        result_number = int(''.join(map(str, [obj[0] for obj in detectedNumbers])))
        result_number = int(result_number)

        return result_number

In [ ]:
#Find a gemstone in the picture. Return: gemstone class, confidence and the stone itself cut out of the image to be given to a classifier later
def getDetectedGemFrameAndPred(frame, model):
    labelsGem, coordinatesGem = detectx(frame, model)
    class_names_Gem = model.names
    for i in range(len(labelsGem)):
        label = labelsGem[i]
        coordinate = coordinatesGem[i]
        if coordinate[4] >= threshold:
            class_label = class_names_Gem[int(label)]
            x1, y1, x2, y2 = map(int,coordinate[:4])
            detected_object = frameGem[y1:y2, x1:x2]

        return detected_object, class_label ,coordinate[4]
    return None, None, None

In [ ]:
#Normalization for image classification
IMAGENET_MEAN = 0.485, 0.456, 0.406
IMAGENET_STD = 0.229, 0.224, 0.225

#Image transform for model
def classify_transforms(size=224):
    return T.Compose([T.ToTensor(), T.Resize(size), T.CenterCrop(size), T.Normalize(IMAGENET_MEAN, IMAGENET_STD)])

In [ ]:
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms as T

#Classification of the given image pieces
def GemClass(model, decObj):
    transformations = classify_transforms()
    converted_tensor = transformations(Image.fromarray(cv2.cvtColor(decObj, cv2.COLOR_BGR2RGB)))
    converted_tensor = converted_tensor.unsqueeze(0)

    results = model(converted_tensor)
    pred = F.softmax(results, dim=1)

    max_confidence, max_class_idx = torch.max(pred, dim=1)
    predGem=model.names[max_class_idx.item()]
    conf=max_confidence.item()

    return predGem, conf

In [ ]:
from IPython.display import clear_output

#Loading models
try:
    modelNumber=loadModel(NumberModelPath)
    modelGem = loadModel(GemModelPath)
    modelGemClass = loadModel(GemModelClassPath)
except:
    print('Error with loading models')

#Setting treshold
global threshold
threshold=0.1

#Reading images
try:

    frameNumber =getFrame(imagePath)
    frameGem = getFrame(imagePath)

except Exception as e:
    print(f'Error reading the image: {str(e)}')


#Detected number
result_number=getDetectedNumber(frameNumber,modelNumber)
#Detected gem
detected_Gem, predObj, confObj=getDetectedGemFrameAndPred(frameGem, modelGem)


if detected_Gem is None:
    clear_output()
    print(f'there is no detected Gem on the picture')
else:
    #Using classification after object detection, to make better prediction
    predClass,confCls =GemClass(modelGemClass, detected_Gem)
    clear_output()
    print(f'Gemstone\'s weight: {result_number}, pred: {predObj}/{predClass}  confidence: {confObj}/{confCls}')

